In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

import os
import acquire
from env import host, user, password

In [2]:
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

# <font color=“brown”>Goal</font>

**My goal is to predict the values of single unit properties using the obervations from 2017**

# Acquisition

In [3]:
def get_connection(db, user=user, host=host, password=password):
    '''
    get_connection uses login info from env.py file to access Codeup db.
    It takes in a string name of a database as an argument.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

**The columns I need**

bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips, propertylandusetypeid

In [4]:
def get_zillow_data():
    '''
    zillow_data() gets the zillow (only properties_2017 table) data from Codeup db, then writes it to a csv file,
    and returns the df.
    '''
    # Creating a SQL query
    sql_query = '''
                SELECT 
                       bedroomcnt,
                       bathroomcnt,
                       calculatedfinishedsquarefeet,
                       taxvaluedollarcnt,
                       yearbuilt,
                       taxamount,
                       fips,
                       propertylandusetypeid
                FROM properties_2017
                JOIN propertylandusetype USING(propertylandusetypeid)
                WHERE propertylandusetypeid = '261'
                '''
    
    # Reading in the DataFrame from Codeup db.
    properties_2017 = pd.read_sql(sql_query, get_connection('zillow'))
    return properties_2017

In [5]:
def get_local_zillow():
    '''
    get_local_zillow reads in telco data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('properties_2017.csv'):
        
        # If csv file exists read in data from csv file.
        properties_2017 = pd.read_csv('properties_2017.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        properties_2017 = get_zillow_data()
        
        # Cache data
        properties_2017.to_csv('properties_2017.csv')
        
    return properties_2017

In [6]:
# Getting a vue of the dataset

properties_2017 = get_zillow_data()
properties_2017.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,261.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0,261.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,261.0


# Cleaning

In [7]:
# Looking for comments that could explain the columns

properties_2017.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 8 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
 7   propertylandusetypeid         float64
dtypes: float64(8)
memory usage: 131.4 MB


In [8]:
# Amount of nulls per columns

properties_2017.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
propertylandusetypeid              0
dtype: int64

In [9]:
# # Identifying the null values inside the columns to make sure they are null values

# properties_2017[['bedroomcnt',
#                  'bathroomcnt',
#                  'calculatedfinishedsquarefeet',
#                  'taxvaluedollarcnt',
#                  'yearbuilt', 'taxamount']].isna()

In [10]:
# Getting the proportions of the missing value in the dataset

round(properties_2017.isnull().mean(), 4)

bedroomcnt                      0.0000
bathroomcnt                     0.0000
calculatedfinishedsquarefeet    0.0039
taxvaluedollarcnt               0.0002
yearbuilt                       0.0043
taxamount                       0.0021
fips                            0.0000
propertylandusetypeid           0.0000
dtype: float64

In [11]:
# Printing the columns that have an obscure meaning to look at their content

properties_2017[['taxvaluedollarcnt', 'taxamount', 'calculatedfinishedsquarefeet']]

,taxvaluedollarcnt,taxamount,calculatedfinishedsquarefeet
0,27516.0,NaN,NaN
1,10.0,NaN,NaN
2,10.0,NaN,NaN
3,2108.0,174.21,NaN
4,296425.0,6941.39,3633.0
...,...,...,...
2152858,960756.0,13494.52,2262.0
2152859,536061.0,6244.16,3127.0
2152860,208057.0,5783.88,NaN
2152861,424353.0,5302.70,1974.0


## Takeaways

<font color="brown">**Based on the quick look on the data, I need to understand what each column means in order to handle them**</font>

**Description of columns**

This gives a description of the dataset columns according to my understanding through examinatiion of the dataset and online research.

    - bedroomcnt: number of bedrooms
    - bathroomcnt: number of bathrooms
    - calculatedfinishedsquarefeet: size of the property in squarefeet
    - taxvaluedollarcnt: the taxable value of a residential property is 100% of its "market value", basically, what it would sell for on the open market
    - yearbuilt: the year the house was built
    - taxamount: the tax to be paid or paid proportionally to the value of the house
    - fips: Federal Information Processing System (number that uniquely identify geographic areas)

**Cleaning process**
* fips and propertylandusetypeid can be dropped because they are identifiers and will not affect the processing of the data
* Some columns (calculatedfinishedsquarefeet, taxvaluedollarcnt, ) need to be renamed for better readability
* The null values all together account for 0.01% of the entire dataset. They can be dropped without affecting modeling
* bedroomcnt, bathroomcnt, yearbuilt are a float and needs to be converted into int64

In [12]:
# Dropping null values
houses = properties_2017.dropna(axis = 0, how ='any')

# Providing the proportion of dropped rows
print("Properties_2017 length:", len(properties_2017), "\nHouses length:", 
       len(houses), "\nPercentage of rows with at least 1 NA value dropped: ",
       round(((len(properties_2017)-len(houses))/len(properties_2017))*100, 2))

Properties_2017 length: 2152863 
Houses length: 2140235 
Percentage of rows with at least 1 NA value dropped:  0.59


In [13]:
# A view of the new dataframe

houses.isna().mean()

bedroomcnt                      0.0
bathroomcnt                     0.0
calculatedfinishedsquarefeet    0.0
taxvaluedollarcnt               0.0
yearbuilt                       0.0
taxamount                       0.0
fips                            0.0
propertylandusetypeid           0.0
dtype: float64

In [14]:
# Dropping columns

houses = houses.drop(["fips", "propertylandusetypeid"], axis = 1)
houses.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68
11,0.0,0.0,1200.0,5328.0,1972.0,91.60
14,0.0,0.0,171.0,6920.0,1973.0,255.17


In [15]:
# Renaming columns using a dictionary
cols_to_rename = {
    'calculatedfinishedsquarefeet': 'squarefeet',
    'taxvaluedollarcnt': 'taxvalue',
}

houses = houses.rename(columns=cols_to_rename)
houses.head()

,bedroomcnt,bathroomcnt,squarefeet,taxvalue,yearbuilt,taxamount
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68
11,0.0,0.0,1200.0,5328.0,1972.0,91.60
14,0.0,0.0,171.0,6920.0,1973.0,255.17


In [16]:
# Checking the values to see if their proportions relate to each other in a meaningful way

houses.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2140235.0,3.301419,0.932600,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2140235.0,2.240352,0.990549,0.00,2.00,2.00,3.00,32.00
squarefeet,2140235.0,1863.193970,1221.754161,1.00,1258.00,1623.00,2208.00,952576.00
taxvalue,2140235.0,460641.625164,677157.635675,22.00,189166.00,328296.00,534606.00,90188462.00
yearbuilt,2140235.0,1960.967545,22.150563,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2140235.0,5616.711322,7814.562798,6.34,2540.85,4111.47,6411.93,1078101.87


In [ ]:
houses.astype(dtype, copy=True, errors='raise')